<a href="https://colab.research.google.com/github/Chu-Yichen/QM2-Group-19/blob/main/Get_Sites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install geopy requests pandas unzip

import pandas as pd
from geopy.distance import geodesic
import requests, zipfile, io

# URL to the zip file
zip_url = "https://aqs.epa.gov/aqsweb/airdata/aqs_monitors.zip"

# Download and extract the zip file
r = requests.get(zip_url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# The main monitor file is typically named 'aqs_monitors.csv'
file_name = 'aqs_monitors.csv'
z.extract(file_name)

# Load the data into a pandas DataFrame
df = pd.read_csv(file_name)

# The state code for Georgia is 13
df_ga = df[df['State Code'] == 13]

# County codes of interest in the Atlanta region
atl_counties = ['057','063','067','089','097','113','117','121','135','151','247']

# Convert the county codes in df_ga to zero-padded strings to match the format
df_ga['county_code_str'] = df_ga['County Code'].astype(str).str.zfill(3)

# Filter by the counties of interest
df_atl = df_ga[df_ga['county_code_str'].isin(atl_counties)]

# Coordinates of Atlanta Airport (Hartsfield–Jackson): (latitude, longitude)
atl_airport_coords = (33.6407, -84.4277)

# Define a function to calculate distance in km using geodesic
def distance_from_airport(lat, lon):
    return geodesic((lat, lon), atl_airport_coords).kilometers

# Calculate the distance of each monitor from the airport
df_atl['distance_km'] = df_atl.apply(lambda row: distance_from_airport(row['Latitude'], row['Longitude']), axis=1)

# Filter monitors within 20 km
within_20km = df_atl[df_atl['distance_km'] <= 20]

# Filter monitors within 30 km (but more than 20 km, if you want separate listings)
within_30km = df_atl[(df_atl['distance_km'] <= 30) & (df_atl['distance_km'] > 20)]

# Print the results
print("Monitors within 20 km of Atlanta Airport:")
print(within_20km[['State Code','County Code','Site Number','Latitude','Longitude','distance_km']])

print("\nMonitors between 20 km and 30 km of Atlanta Airport:")
print(within_30km[['State Code','County Code','Site Number','Latitude','Longitude','distance_km']])


<ipython-input-15-3478d9c7a902>:22: DtypeWarning: Columns (0,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)
<ipython-input-15-3478d9c7a902>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ga['county_code_str'] = df_ga['County Code'].astype(str).str.zfill(3)


Monitors within 20 km of Atlanta Airport:
      State Code  County Code  Site Number   Latitude  Longitude  distance_km
84006         13           63            1  33.603201 -84.390195     5.423162
84007         13           63            1  33.603201 -84.390195     5.423162
84008         13           63           91  33.610845 -84.390717     4.768752
84009         13           63           91  33.610845 -84.390717     4.768752
84010         13           63           91  33.610845 -84.390717     4.768752
...          ...          ...          ...        ...        ...          ...
86315         13          121         5002  33.699552 -84.447151     6.772319
86316         13          121         5002  33.699552 -84.447151     6.772319
86317         13          121         5002  33.699552 -84.447151     6.772319
86318         13          121         5002  33.699552 -84.447151     6.772319
86319         13          121         5002  33.699552 -84.447151     6.772319

[1415 rows x 6 column

<ipython-input-15-3478d9c7a902>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atl['distance_km'] = df_atl.apply(lambda row: distance_from_airport(row['Latitude'], row['Longitude']), axis=1)
